Import potrebnih knjižnic za analizo podatkov:

In [59]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Prebiranje datotek s podakti, tukaj se ignorira vrstice, ki pokvarijo strukturo podatkov oziroma v originalni datoteki niso del podatkov temvec prazna vrstica ipd. 
V programski kodi so seveda zamaknjene za -1.

Izberemo tudi samo stolpce z izbranimi točkami:  
A - datumi  
B - povprecje  
C:IW - izbrane tocke  


od IW naprej so pa stolpci prazni in nekje v koncu dokumenta spet nekatere vrednosti, ki niso pomembne

Stolpec z datumi se spremeni v datume.

In [60]:
file_path = 'podatki/Rizana_Zaledje_INCA_dnev_2020_2021_N1.xlsx'
df_padavine = pd.read_excel(file_path, sheet_name='Izbrane', skiprows=[0, 1, 3, 4], usecols="A,C:IW")

df_padavine["Datum"] = pd.to_datetime(df_padavine["Datum"], format="%m/%d/%y")
df_padavine = df_padavine.set_index('Datum')


preberemo se za pretok, po stoplcih:  
A - datum  
B - pretok  
C - prazen  
D - temp. vode  
E - mot. vode  

se preimenujem pretok da bo lazje obdelovati

In [61]:
file_path1 = 'podatki/Pretok_2020_2021_N1.xlsx'
df_pretok = pd.read_excel(file_path1, sheet_name='meritve', usecols="A, B")
df_pretok.rename(columns={"pretok (m3/s)": "pretok"}, inplace=True)

df_pretok["Datum"] = pd.to_datetime(df_pretok["Datum"], format="%d.%m.%Y")
df_pretok = df_pretok.set_index('Datum')


zdrzuimo oba dataframa s concat (axis = 1 pomeni da po stolpcih)

#### Normalne vrednosti in % odstopanja
- pogledati je treba katere so normalne vrednosti in kakšen procent predstavlja znatno odstopanje

In [62]:
df = pd.concat([df_pretok, df_padavine], axis=1)

print(f"Velikost: {df.shape}")
print(f"Stolpci: {df.columns.tolist()}")

x = 10
#print(df_pretok.head(x))
#print(df_padavine.head(x))
#print(df.sample(x))

print(f"Toliko vrstic ima Nan: {df.isna().any(axis=1).sum()}")

avgPretok = df['pretok'].mean().round(3) #zaokroženo na 3 mesta kot podatki
modPretok = df["pretok"].mode().iloc[0]
medPretok = df["pretok"].median().round(3)

print(f"Povprečni pretok: {avgPretok}")
print(f"Mediana pretok: {medPretok}")
print(f"Najbolj pogost pretok: {modPretok}") #mode vrne vecje stevilo zato treba z iloc preveriti prvega ne glede na index

Velikost: (731, 256)
Stolpci: ['pretok', '68', '69', '95', '96', '97', '98', '122', '123', '124', '125', '126', '127', '128', '150', '151', '152', '153', '154', '155', '156', '157', '158', '178', '179', '180', '181', '182', '183', '184', '185', '186', '206', '207', '208', '209', '210', '211', '212', '213', '214', '218', '219', '230', '231', '235', '236', '237', '238', '239', '240', '241', '242', '243', '244', '245', '246', '247', '248', '258', '259', '260', '261', '262', '263', '264', '265', '266', '267', '268', '269', '270', '271', '272', '273', '274', '275', '276', '277', '286', '287', '288', '289', '290', '291', '292', '293', '294', '295', '296', '297', '298', '299', '300', '301', '302', '303', '304', '305', '314', '315', '316', '317', '318', '319', '320', '321', '322', '323', '324', '325', '326', '327', '328', '329', '330', '331', '332', '333', '343', '344', '345', '346', '347', '348', '349', '350', '351', '352', '353', '354', '355', '356', '357', '358', '359', '360', '361', '371',

Prikaz vrednosti s stacked bar grafom, večina kode napisana s pomočjo AI orodji. Cilj je razumeti podatke!